In [3]:
from __future__ import annotations

from pydantic import BaseModel

from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    OpenAIChatCompletionsModel
)

from openai import AsyncAzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

"""
This example shows how to use guardrails.

Guardrails are checks that run in parallel to the agent's execution.
They can be used to do things like:
- Check if input messages are off-topic
- Check that output messages don't violate any policies
- Take over control of the agent's execution if an unexpected input is detected

In this example, we'll setup an input guardrail that trips if the user is asking to do math homework.
If the guardrail trips, we'll respond with a refusal message.
"""


# 1. An agent-based guardrail that is triggered if the user is asking to do math homework
class MathHomeworkOutput(BaseModel):
  reasoning: str
  is_math_homework: bool


guardrail_agent = Agent(
    name="Guardrail check",
    model=OpenAIChatCompletionsModel(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        openai_client=client,
    ),
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(
    context: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
  """This is an input guardrail function, which happens to call an agent to check if the input
  is a math homework question.
  """
  result = await Runner.run(guardrail_agent, input, context=context.context)
  final_output = result.final_output_as(MathHomeworkOutput)

  return GuardrailFunctionOutput(
      output_info=final_output,
      tripwire_triggered=final_output.is_math_homework,
  )


# 2. The run loop


async def main():
  agent = Agent(
      name="Customer support agent",
      model=OpenAIChatCompletionsModel(
          model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
          openai_client=client,
      ),
      instructions="You are a customer support agent. You help customers with their questions.",
      input_guardrails=[math_guardrail],
  )

  input_data: list[TResponseInputItem] = []

  while True:
    user_input = input("Enter a message: ")
    input_data.append(
        {
            "role": "user",
            "content": user_input,
        }
    )

    try:
      result = await Runner.run(agent, input_data)
      print(result.final_output)
      # If the guardrail didn't trigger, we use the result as the input for the next run
      input_data = result.to_input_list()
    except InputGuardrailTripwireTriggered:
      # If the guardrail triggered, we instead add a refusal message to the input
      message = "Sorry, I can't help you with your math homework."
      print(message)
      input_data.append(
          {
              "role": "assistant",
              "content": message,
          }
      )

  # Sample run:
  # Enter a message: What's the capital of California?
  # The capital of California is Sacramento.
  # Enter a message: Can you help me solve for x: 2x + 5 = 11
  # Sorry, I can't help you with your math homework.


if __name__ == "__main__":
  await main()

Seattle is a major city located in the state of Washington, in the United States. It is situated in the Pacific Northwest region, along the western coast of the country. Seattle lies between Puget Sound and Lake Washington, and it is known for its vibrant tech industry, iconic Space Needle, and beautiful natural surroundings. If you need more specific information or directions, feel free to ask!
Sorry, I can't help you with your math homework.
Sorry, I can't help you with your math homework.
Sorry, I can't help you with your math homework.
Sorry, I can't help you with your math homework.
If you need any further assistance, feel free to reach out. Have a great day!
Sorry, I can't help you with your math homework.
Sorry, I can't help you with your math homework.
Sorry, I can't help you with your math homework.


CancelledError: 